# RQ2 (N=160, All 19 Properties)
This notebook rebuilds RQ2 over all 160 reports using **all 19 canonical properties**. It augments the base CSV with text-derived signals parsed from the enrichment spreadsheets (Report Structure, Methodology, Severity definitions, Knowledge) to recover additional sections, runs FCA over the full set, mines implication rules, defines style families, and exports CSVs and a providers×families heatmap.

> Note: `Detailed Findings` is universal in this run and is **not** used to define families.

In [ ]:

import pandas as pd, numpy as np, re
from pathlib import Path
import matplotlib.pyplot as plt
from itertools import combinations

base = Path("/mnt/data")
out = base / "rq2_160_all19_outputs"
out.mkdir(parents=True, exist_ok=True)

# === Load base 160 CSV
base_df = pd.read_csv(base / "all_reports_coded_with_links.csv")
base_df.columns = [c.strip() for c in base_df.columns]
for col in base_df.columns:
    if col in ["Provider","Project","Link","Coder"]:
        continue
    base_df[col] = pd.to_numeric(base_df[col], errors="coerce").fillna(0).astype(int)

# === Load enrichment sheets
def load_sheet(path):
    xls = pd.ExcelFile(path)
    df = pd.read_excel(xls, sheet_name=xls.sheet_names[0])
    df.columns = [str(c).strip() for c in df.columns]
    return df

xlsx_paths = [
    base / "audit_reports_extraction.xlsx",
    base / "audit_reports_extracted.xlsx",
    base / "remaining_audit_reports_extraction.xlsx",
]
extra_frames = []
for p in xlsx_paths:
    if p.exists():
        extra_frames.append(load_sheet(p))

def unify_cols(df):
    df = df.copy()
    if "Project Name" in df.columns and "Project_Name" not in df.columns:
        df["Project_Name"] = df["Project Name"]
    if "Link" not in df.columns:
        for c in df.columns:
            if c.lower() in ["url","report link","report_url"]:
                df["Link"] = df[c]; break
    for col in ["Provider","Project_Name","Link","Report Structure","Report_Structure","Knowledge","Methodology","Severity Level Definitions","Severity_Level_Definitions"]:
        if col not in df.columns:
            df[col] = ""
    return df

def row_text_blob(row):
    parts = []
    for c in ["Report Structure","Report_Structure","Knowledge","Methodology","Severity Level Definitions","Severity_Level_Definitions"]:
        v = str(row.get(c,""))
        if pd.notna(v) and v.strip():
            parts.append(v)
    return " | ".join(parts).lower()

extras = []
for df in extra_frames:
    dfu = unify_cols(df)
    dfu["__text__"] = dfu.apply(row_text_blob, axis=1)
    dfu["Link_key"] = dfu["Link"].astype(str).str.strip().str.lower()
    dfu["Provider_key"] = dfu["Provider"].astype(str).str.strip().str.lower()
    dfu["Project_key"]  = dfu["Project_Name"].astype(str).str.strip().str.lower()
    extras.append(dfu)

base_df["Link_key"]    = base_df["Link"].astype(str).str.strip().str.lower()
base_df["Provider_key"] = base_df["Provider"].astype(str).str.strip().str.lower()
base_df["Project_key"]  = base_df["Project"].astype(str).str.strip().str.lower()

# === Patterns for all 19 properties (Detailed Findings set to 1 later)
pat = {
    "Executive Summary": r"\bexecutive (summary|overview)\b|technical summary",
    "Scope": r"\bscope\b|out of scope",
    "System/Protocol Overview": r"\b(system|protocol|project)\s+(overview|architecture|design)|\boverview\b",
    "Versioning & References": r"\bcommit\b|\brevision\b|\bhash\b|\brepository\b|\bversion\b|\breferences\b|\bref\.\b|appendix",
    "Methodology / Procedure": r"\bmethodology\b|\bmethods?\b|\bprocedure\b|\bapproach\b",
    "Tools & Automation": r"\btool(s|ing)?\b|slither|mythril|oyente|manticore|echidna|foundry|halmos|static analysis|automation",
    "Severity Model / Risk Classification": r"\bseverity (definitions?|matrix|categories)\b|risk classification|cvss|bvss",
    "Findings Summary (Table)": r"\bfindings (summary|table)\b|summary table|vulnerability summary|severity table",
    "Recommendations / Remediation Guidance": r"\brecommend(ation|ations|ed|s)\b|\bmitigat(e|ion|ions)\b|\bremediation\b|\bsuggest(s|ed|ions?)\b",
    "Status of Findings / Verification": r"\bstatus\b|fixed|partially fixed|acknowledged|verification|re-?check|final review|post-?audit|fix review",
    "Threat / Trust Model": r"\bthreat model\b|\btrust model\b|\bassumption(s)?\b|\bsecurity specification\b",
    "Privileged Roles & Centralisation Risks": r"\bprivileg(e|ed)\b|admin key|owner privileg|centralis(z|s)ation|centraliz(s|z)ation|centralization risk",
    "Risk Taxonomy / Terminology & References": r"\bterminology\b|\btaxonomy\b|\bglossary\b|\bdefinitions?\b|\bsecurity specification\b",
    "Timeline / Engagement Details": r"\btimeline\b|engagement (timeline|period)|review period|start date|end date",
    "Testing / Coverage Quality": r"\bcoverage\b|unit testing|tests? coverage|test quality|project coverage|fuzz(ing)? coverage",
    "Documentation Quality": r"\bdocumentation\b.*\bquality\b|documentation quality|docs quality|documentation assessment",
    "Upgradeability / Governance Controls": r"\bgovernance\b|upgradeab(le|ility)|upgradeable|proxy|time[- ]?lock|governance controls",
    "Limitations / Disclaimers / Non-Goals": r"\bdisclaimer(s)?\b|limitation(s)?|non[- ]?goals|not a guarantee|no warranty",
}

props19 = [
    'Executive Summary','Scope','System/Protocol Overview','Versioning & References',
    'Methodology / Procedure','Tools & Automation','Severity Model / Risk Classification',
    'Findings Summary (Table)','Detailed Findings','Recommendations / Remediation Guidance',
    'Status of Findings / Verification','Threat / Trust Model',
    'Privileged Roles & Centralisation Risks','Risk Taxonomy / Terminology & References',
    'Timeline / Engagement Details','Testing / Coverage Quality','Documentation Quality',
    'Upgradeability / Governance Controls','Limitations / Disclaimers / Non-Goals'
]

def parse_flags(text):
    flags = {}
    for p, rgx in pat.items():
        flags[p] = 1 if re.search(rgx, text, flags=re.I) else 0
    return flags

parsed_records = []
for dfu in extras:
    for _, row in dfu.iterrows():
        text = row["__text__"]
        flags = parse_flags(text)
        rec = {"Link_key": row["Link_key"], "Provider_key": row["Provider_key"], "Project_key": row["Project_key"]}
        rec.update(flags)
        parsed_records.append(rec)

parsed_df = pd.DataFrame(parsed_records) if parsed_records else pd.DataFrame(columns=["Link_key","Provider_key","Project_key"]+list(pat.keys()))
agg_by_link = parsed_df.groupby("Link_key").max().reset_index()
agg_by_pp   = parsed_df.groupby(["Provider_key","Project_key"]).max().reset_index()

merged = base_df.merge(agg_by_link, on="Link_key", how="left", suffixes=("","_parsed"))
merged = merged.merge(agg_by_pp, on=["Provider_key","Project_key"], how="left", suffixes=("","_pp"))

final = pd.DataFrame(index=merged.index)
final["Provider"] = merged["Provider"]
final["Project"]  = merged["Project"]
final["Link"]     = merged["Link"]

base_map = {
    "Executive Summary": ["Summary"],
    "Scope": ["Scope"],
    "Methodology / Procedure": ["Methodology"],
    "Tools & Automation": ["Automatic Detection","Detection Tools","Techniques Used Mentions"],
    "Recommendations / Remediation Guidance": ["Recommendations"],
    "Status of Findings / Verification": ["Fix Review Included","Status of Findings"],
    "Testing / Coverage Quality": ["Test Quality"],
    "Documentation Quality": ["Documentation Quality"],
    "Limitations / Disclaimers / Non-Goals": ["Limitations Mentioned"],
}

for p in props19:
    if p == "Detailed Findings":
        final[p] = 1
        continue
    base_val = 0
    for src in base_map.get(p, []):
        if src in merged.columns:
            base_val = base_val | merged[src].fillna(0).astype(int)
    parsed_link_col = p
    parsed_pp_col   = p + "_pp"
    if parsed_link_col not in merged.columns: merged[parsed_link_col] = 0
    if parsed_pp_col not in merged.columns:   merged[parsed_pp_col] = 0
    final[p] = (base_val | merged[parsed_link_col].fillna(0).astype(int) | merged[parsed_pp_col].fillna(0).astype(int)).astype(int)

context19 = final.copy()
context19.to_csv(out/"RQ2_FCA_context_160_all19.csv", index=False)

# --- Prevalence
attr19 = pd.DataFrame({"Property": props19, "Support": [int(context19[p].sum()) for p in props19]})
attr19["Coverage(%)"] = (attr19["Support"]/len(context19)*100).round(1)
attr19 = attr19.sort_values("Support", ascending=False)
attr19.to_csv(out/"RQ2_attribute_support_coverage_160_all19.csv", index=False)

# --- FCA (Next-Closure)
A = props19
obj_attrs_all = [{a for a in A if context19.loc[i, a]==1} for i in context19.index]

def extent_of_intent(intent):
    return set(i for i,attrs in enumerate(obj_attrs_all) if intent.issubset(attrs))
def intent_of_extent(ext):
    if not ext: return set(A)
    inter = set(A)
    for i in ext: inter &= obj_attrs_all[i]
    return inter
def closure_intent(X): return intent_of_extent(extent_of_intent(X))

def next_closure(current):
    curr_bits = [1 if a in current else 0 for a in A]
    for i in range(len(A)-1, -1, -1):
        if curr_bits[i]==0:
            y = set(current); y.add(A[i]); y = closure_intent(y)
            ok = True
            for j in range(i):
                if (A[j] in y) != (curr_bits[j]==1): ok = False; break
            if ok: return y
    return None

concepts = []; seen = set()
X = closure_intent(set())
while True:
    key = tuple(sorted(X))
    if key not in seen:
        seen.add(key)
        ext = extent_of_intent(X)
        concepts.append({"IntentSize": len(X), "Support": len(ext), "Intent": "; ".join(sorted(X))})
    nxt = next_closure(X)
    if nxt is None or nxt == X: break
    X = nxt

concepts_all19_df = pd.DataFrame(concepts).sort_values(["Support","IntentSize"], ascending=[False,True]).reset_index(drop=True)
concepts_all19_df.to_csv(out/"RQ2_FCA_concepts_160_all19.csv", index=False)

# --- Implications and DG-like pruned set
impl = []
def sup(S): return len(extent_of_intent(set(S)))
for r in range(1,4):
    for comb in combinations(A, r):
        c = closure_intent(set(comb))
        if set(c) > set(comb):
            impl.append({"Premise": "; ".join(comb), "Implied": "; ".join(sorted(set(c)-set(comb))), "Support": sup(comb)})
imp_all19_df = pd.DataFrame(impl).sort_values("Support", ascending=False).reset_index(drop=True)
imp_all19_df.to_csv(out/"RQ2_FCA_implications_160_all19.csv", index=False)

imp_all19_df["Pset"] = imp_all19_df["Premise"].apply(lambda s: frozenset(x.strip() for x in s.split(";")))
imp_all19_df["Cset"] = imp_all19_df["Implied"].apply(lambda s: frozenset(x.strip() for x in s.split(";") if x.strip()))
keep = [True]*len(imp_all19_df)
for i in range(len(imp_all19_df)):
    if not keep[i]: continue
    Pi, Ci = imp_all19_df.loc[i,"Pset"], imp_all19_df.loc[i,"Cset"]
    for j in range(i+1, len(imp_all19_df)):
        if not keep[j]: continue
        Pj, Cj = imp_all19_df.loc[j,"Pset"], imp_all19_df.loc[j,"Cset"]
        if Pi.issubset(Pj) and Cj.issubset(Ci): keep[j] = False
dg_like_all19 = imp_all19_df.loc[keep, ["Premise","Implied","Support"]].reset_index(drop=True)
dg_like_all19.to_csv(out/"RQ2_DG_canonical_basis_like_160_all19.csv", index=False)

# --- Families
families_all19 = {
    "Executive-Packaged": {"all_of": ["Executive Summary"], "any_of": ["Severity Model / Risk Classification","Findings Summary (Table)"]},
    "Core-Engineering": {"all_of": ["System/Protocol Overview"], "any_of": ["Methodology / Procedure","Tools & Automation","Testing / Coverage Quality"]},
    "Remediation-First": {"all_of": [], "any_of": ["Recommendations / Remediation Guidance","Status of Findings / Verification"]},
    "Governance-Focused": {"all_of": [], "any_of": ["Threat / Trust Model","Privileged Roles & Centralisation Risks","Upgradeability / Governance Controls"]},
    "Legal/Taxonomy-Heavy": {"all_of": ["Limitations / Disclaimers / Non-Goals"], "any_of": ["Risk Taxonomy / Terminology & References","Severity Model / Risk Classification"]}
}
def fam_memberships_all19(row):
    outm = {}
    for fam, rule in families_all19.items():
        all_ok = all(row.get(a,0)==1 for a in rule["all_of"])
        any_ok = (len(rule["any_of"])==0) or any(row.get(a,0)==1 for a in rule["any_of"])
        outm[fam] = int(all_ok and any_ok)
    return pd.Series(outm)

fam_mat_19 = context19.apply(fam_memberships_all19, axis=1)
fam_cov_19 = fam_mat_19.sum().to_frame("ReportsWithFamily")
fam_cov_19["Coverage(%)"] = (fam_cov_19["ReportsWithFamily"]/len(context19)*100).round(1)
fam_cov_19 = fam_cov_19.sort_values("ReportsWithFamily", ascending=False)
fam_cov_19.reset_index().rename(columns={"index":"Family"}).to_csv(out/"RQ2_family_coverage_160_all19.csv", index=False)

prov_soft_19 = pd.concat([context19[["Provider","Project"]], fam_mat_19], axis=1)
prov_soft_ratios_19 = prov_soft_19.groupby("Provider")[list(families_all19.keys())].mean().round(2)
prov_soft_counts_19 = prov_soft_19.groupby("Provider").size().to_frame("N_reports")
prov_soft_table_19 = prov_soft_counts_19.join(prov_soft_ratios_19)
prov_soft_table_19.to_csv(out/"RQ2_provider_family_soft_160_all19.csv")

plt.figure(figsize=(10, max(4, 0.35*len(prov_soft_ratios_19))))
plt.imshow(prov_soft_ratios_19.values, aspect='auto')
plt.xticks(range(len(prov_soft_ratios_19.columns)), prov_soft_ratios_19.columns, rotation=45, ha='right')
plt.yticks(range(len(prov_soft_ratios_19.index)), prov_soft_ratios_19.index)
plt.colorbar()
plt.title("Soft adoption of style families by provider (N=160, all 19 properties)")
plt.tight_layout()
plt.savefig(out/"figure_soft_adoption_heatmap_160_all19.png", dpi=200)
plt.close()
